<a href="https://colab.research.google.com/github/VLCarolina/Flowers/blob/main/Flowers_Alexnet_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from google.colab import drive
from torchsummary import summary

In [2]:
# 1. Mount Google Drive
drive.mount('/content/gdrive')

# 2. Load labeled images from folders
# data_dir = '/content/gdrive/MyDrive/Data1/antsbeesdataset'
data_dir = '/content/gdrive/MyDrive/Data2023/Flowers'

Mounted at /content/gdrive


In [3]:
cd /content/gdrive/MyDrive/Data2023/Flowers /

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/Data2023/Flowers /'
/content


In [4]:
ls

gdrive/  sample_data/


In [5]:
# 3. Pre-process the data and create data loaders
data_transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
image_datasets = {x: datasets.ImageFolder(data_dir + '/' + x, data_transforms[x]) for x in ['Train', 'Valid']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['Train', 'Valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['Train', 'Valid']}
class_names = image_datasets['Train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
# 4. Set up the AlexNet architecture
alexnet = models.alexnet(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 236MB/s]


In [8]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [9]:

num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names)) # Change last layer
alexnet = alexnet.to(device) # Put on GPU

In [10]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [11]:
summary(alexnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [12]:
class_names

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [13]:
dataset_sizes

{'Train': 125, 'Valid': 34}

In [14]:
for inputs, labels in dataloaders["Train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

In [15]:
inputs.shape

torch.Size([13, 3, 224, 224])

In [16]:
labels.shape

torch.Size([13])

In [17]:
labels

tensor([16,  1, 20, 23, 11, 11,  7, 27, 28, 23, 24, 14, 19], device='cuda:0')

In [18]:
outputs = alexnet(inputs)

In [19]:
outputs.shape

torch.Size([13, 30])

In [20]:
outputs

tensor([[ 0.1063,  0.0806, -0.2002,  0.2300,  0.4307,  0.2919, -0.5901, -1.0930,
          0.0164,  1.1225,  1.0919, -0.3575,  0.1014,  0.7508,  0.6840,  0.7922,
         -0.2078, -0.3326,  1.5976,  0.0222,  0.2853, -0.7379,  0.3903,  0.0478,
         -0.0617,  0.2313,  0.6261, -0.8001,  0.0186,  1.1181],
        [ 0.5188, -0.2837, -0.0815, -0.5249, -0.2455, -0.4923, -0.4099,  0.8123,
         -0.4956,  0.2667,  0.5897,  0.6549,  0.9006, -0.1941,  0.4036,  0.1499,
          0.3228, -0.3753,  0.8236, -0.3276, -0.5814,  0.0745, -0.2366, -0.1734,
          0.5259,  0.0801, -0.0950,  0.4672,  0.4342, -0.1169],
        [ 1.1007, -0.2889, -0.0266, -0.1986, -0.2133,  0.2631, -0.1012,  0.0138,
          0.3915,  0.9309, -0.0243,  0.3354, -0.1993,  0.8798, -1.1821, -1.0113,
          1.0864, -0.0711,  0.6508,  0.0377, -0.8748, -0.3824,  0.1558,  0.0226,
          0.7552, -0.5756, -0.6829, -1.2185,  0.3068,  0.3403],
        [-0.5896,  0.7381,  0.2159,  0.2786,  0.2851,  2.6034, -0.5384, -0.6959

In [21]:
torch.max(outputs,1)[0]

tensor([1.5976, 0.9006, 1.1007, 2.6034, 1.4664, 1.2598, 1.3606, 1.2583, 2.3216,
        1.7181, 1.2508, 1.4351, 1.0805], device='cuda:0',
       grad_fn=<MaxBackward0>)

In [22]:
torch.max(outputs,1)[1]

tensor([18, 12,  0,  5, 18, 18,  0, 16,  0,  5,  7,  6,  7], device='cuda:0')

In [23]:
preds = torch.max(outputs, 1)[1]

In [24]:
preds

tensor([18, 12,  0,  5, 18, 18,  0, 16,  0,  5,  7,  6,  7], device='cuda:0')

In [25]:
labels

tensor([16,  1, 20, 23, 11, 11,  7, 27, 28, 23, 24, 14, 19], device='cuda:0')

In [26]:
preds == labels

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False], device='cuda:0')

In [27]:
labels.shape[0]

13

In [28]:
torch.sum(preds == labels)/labels.shape[0]

tensor(0., device='cuda:0')

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# Train the AlexNet model
alexnet.train()

num_epochs = 20

for epoch in range(num_epochs):
    print(epoch, " of ", num_epochs - 1)
    print('-' * 10)

    running_corrects = 0

    for inputs, labels in dataloaders["Train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        

        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]



        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_corrects += torch.sum(preds == labels.data)

    print('Train Acc: {:.4f}'.format(running_corrects / dataset_sizes["Train"]))




    # Evaluate the AlexNet model on Validation Data
    alexnet.eval()

    running_corrects = 0

    for inputs, labels in dataloaders["Valid"]:
        inputs = inputs.to(device)
        labels = labels.to(device)


        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]


        running_corrects += torch.sum(preds == labels.data)

    acc_valid = running_corrects / dataset_sizes["Valid"]
    print('Valid Acc: {:.4f}'.format(acc_valid))
    if acc_valid > 0.99:
        print("Done!")
        break


print('Training complete')

0  of  19
----------
Train Acc: 0.0880
Valid Acc: 0.5294
1  of  19
----------
Train Acc: 0.6000
Valid Acc: 0.8824
2  of  19
----------
Train Acc: 0.9200
Valid Acc: 0.8824
3  of  19
----------
Train Acc: 0.8400
Valid Acc: 0.9118
4  of  19
----------
Train Acc: 0.9040
Valid Acc: 0.9412
5  of  19
----------
Train Acc: 0.9120
Valid Acc: 1.0000
Done!
Training complete


In [30]:
pwd

'/content'

In [31]:
model_path = '/content/gdrive/MyDrive/Data2023/SmartTrashCan/alexnet_classification_SmartTrashCan.pth'

In [32]:
# Save the trained model
torch.save(alexnet.state_dict(), model_path)

In [33]:
ls

gdrive/  sample_data/


In [34]:
import requests
from PIL import Image
from io import BytesIO

# 1. Load the trained AlexNet model
def load_model(model_path):
    model = models.alexnet()
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model.to(device)

# 2. Define a function to load an image from a URL and preprocess it
def preprocess_image(url, transform):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img_tensor = transform(img)
    return img_tensor.unsqueeze(0).to(device)

# 3. Perform inference using the loaded model
def predict_image_url(url, model):
    img_tensor = preprocess_image(url, data_transforms['Valid'])
    output = model(img_tensor)
    pred = torch.max(output, 1)[1]
    return class_names[pred]

In [35]:
trained_model = load_model(model_path)

In [36]:
image_url = 'https://agrogojarviveros.com/entradavivero/uploads/2018/08/orqu%C3%ADdea.jpg'

In [37]:
prediction = predict_image_url(image_url, trained_model)

In [38]:
print("The predicted class for the input image is:", prediction)

The predicted class for the input image is: 25
